### VGGNet
- 2014년 ILSVRC에서 GoogleNet이 1위, VGG는 2위를 차지했다.
- GoogleNet의 오류율은 6.7%, VGG의 오류율은 7.3%이고, 0.6% 차이밖에 나지 않았다.
- 간결하고 단순한 아키텍처임에도 불구하고 1위인 GoogleNet과 큰 차이 없는 성능을 보여서 주목을 받게 되었다.
- 네트워크의 깊이에 따른 모델 성능의 영향에 대한 연구에 집중하여 만들어진 네트워크이다.
- 신경망을 깊게 만들 수록 성능이 좋아짐을 확인하였지만, 커널 사이즈가 클 수록 이미지 사이즈가 급격하게 축소되기 때문에  
더 깊은 층을 만들기 어렵고, 파라미터 개수와 연산량도 더 많이 필요하다는 것을 알았다.
- 따라서 Kernel 크기를 3x3으로 단일화했으며, Padding, Strides 값을 조정하여 단순한 네트워크로 구성되었다.
- 2개의 3x3 커널은 5x5 커널과 동일한 크기의 Feature map 생성하기 때문에 3x3 커널로 연산하면, 층을 더 만들 수 있게 된다.
- AlexNet보다 더 깊은 Network을 구성하였음에도 더 적은 Parameter 개수로 연산 성능을 개선시켰다.
- 영상에서 특정 위치에 있는 픽셀들은 그 주변에 있는 일부 픽셀들 하고만 상관관계가 높고 거리가 멀어질수록 그 영향은 감소한다.
- 위의 성질을 통해, 전체 영역을 처리하는 대신에 특정 범위를 한정해 처리를 하면 훨씬 효과적이며, 이 영역을 Receptive Field(수용 영역)이라고 한다.
- 즉, Receptive Field는 입력 데이터에서 Feature를 만드는 영역의 기본 크기를 의미한다.

<img src="./images/VGG.png" style='margin-left: 0'>

In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau , EarlyStopping , ModelCheckpoint

def vggnet(input_shape=(224, 224, 3), n_classes=10):
    input_tensor = Input(shape=input_shape)
    
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(input_tensor)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(units = 120, activation = 'relu')(x)
    x = Dropout(0.5)(x)

    output = Dense(units = n_classes, activation = 'softmax')(x)
    
    model = Model(inputs=input_tensor, outputs=output)
    model.summary()
    
    return model

In [2]:
model = vggnet(input_shape=(224, 224, 3), n_classes=10)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     